In [2]:
pip install simplet5

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd

In [4]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [13]:
input_dat = "/kaggle/input/newspaper-text-summarization-cnn-dailymail/cnn_dailymail/train.csv"

In [16]:
df = pd.read_csv(input_dat, encoding='latin-1')
df.head()

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...
2,00027e965c8264c35cc1bc55556db388da82b07f,A drunk driver who killed a young woman in a h...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,0002c17436637c4fe1837c935c04de47adb18e9a,(CNN) -- With a breezy sweep of his pen Presid...,Nina dos Santos says Europe must be ready to a...
4,0003ad6ef0c37534f80b55b4235108024b407f0b,Fleetwood are the only team still to have a 10...,Fleetwood top of League One after 2-0 win at S...


In [17]:
df = df.rename(columns={"highlights":"target_text", "article":"source_text"})
df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: since it is a summarization task, we will add a prefix "summarize: "
df['source_text'] = "summarize: " + df['source_text']
df

,source_text,target_text
0,summarize: By . Associated Press . PUBLISHED: ...,"Bishop John Folda, of North Dakota, is taking ..."
1,summarize: (CNN) -- Ralph Mata was an internal...,Criminal complaint: Cop used his role to help ...
2,summarize: A drunk driver who killed a young w...,"Craig Eccleston-Todd, 27, had drunk at least t..."
3,summarize: (CNN) -- With a breezy sweep of his...,Nina dos Santos says Europe must be ready to a...
4,summarize: Fleetwood are the only team still t...,Fleetwood top of League One after 2-0 win at S...
...,...,...
287108,summarize: By . James Rush . Former first daug...,Chelsea Clinton said question of running for o...
287109,summarize: An apologetic Vanilla Ice has given...,"Vanilla Ice, 47 - real name Robert Van Winkle ..."
287110,summarize: America's most lethal sniper claime...,America's most lethal sniper made comment in i...
287111,summarize: By . Sara Malm . PUBLISHED: . 12:19...,A swarm of more than one million has crossed b...


In [19]:
summary_model

0     Bishop John Folda, of North Dakota, is taking ...
1     Criminal complaint: Cop used his role to help ...
2     Craig Eccleston-Todd, 27, had drunk at least t...
3     Nina dos Santos says Europe must be ready to a...
4     Fleetwood top of League One after 2-0 win at S...
                            ...                        
95    Netherlands, Belgium and Qatar all had monarch...
96    Zhu Sanni, 23, had been left alone at home for...
97    Official: Bottles are almost exclusively from ...
98    Two skeletons were found holding hands, buried...
99    Danny Cevallos: Arrests over tweets threatenin...
Name: target_text, Length: 100, dtype: object

In [8]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2)
train_df.shape, test_df.shape

((229690, 2), (57423, 2))

In [9]:
from simplet5 import SimpleT5
model = SimpleT5()


In [10]:
model.from_pretrained(model_type="t5", model_name="t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [11]:

model.train(train_df=train_df[:20000], 
            eval_df=test_df[:300],  
            source_max_token_len=300, 
            target_max_token_len=100, 
            batch_size=16, max_epochs=2, use_gpu=True, 
            early_stopping_patience_epochs=2)

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [5]:
import torch

In [14]:
filepath = "model.pt"

# save the model
torch.save(model, filepath)

In [6]:
model1 = torch.load('model.pt', map_location=torch.device('cpu'))


In [7]:
model1.device= torch.device("cpu")

In [8]:
text_input = """A 38-year-old woman in the US, who was apprehended twice for allegedly trying to jump the White House fence last week, has been arrested for scaling a fence at the Treasury Building. Marci Anderson Wahl of Everett, Washington, was arrested after an alarm sounded at about 2:15 am yesterday when she scaled a fence at the Treasury Building, next to the White House. Police said Wahl has told them she was there to speak to US President Donald Trump, the CNN reported. She was charged with unlawful entry and contempt of court. Wahl was first arrested on March 21 last week for trying to jump the White House fence. Once in custody, it was determined that Wahl had been issued a "stay away" order for the White House complex after the incident. On March 21, officers saw Wahl walking and staring at the White House complex before discovering she had jumped a fence on the south side but got stuck. Officers found her hanging from the inside of the fence by her shoelaces, which were "caught on top of the fence," according to a police report. She was re-arrested on March 24 after officers saw her near Lafayette Park. She was released on her own recognisance after a Saturday court appearance. In the incident, Wahl was charged with contempt of court in violation of a stay away order. She pleaded not guilty last Saturday and was again released on her own recognisance."""


In [9]:
text_to_summarize = """summarize: """ + text_input
predicted_output = model1.predict(text_to_summarize)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
predicted_output

['Marci Anderson Wahl, 38, was apprehended twice for allegedly trying to jump the White House fence. She scaled a fence at the Treasury Building, next to the White House. Police said she told them she was there to speak to US President Donald Trump.']

In [22]:
text_summaries = []
model_summaries = []

In [24]:
summary_model = df['target_text'][:20]

In [25]:
summary_model

0     Bishop John Folda, of North Dakota, is taking ...
1     Criminal complaint: Cop used his role to help ...
2     Craig Eccleston-Todd, 27, had drunk at least t...
3     Nina dos Santos says Europe must be ready to a...
4     Fleetwood top of League One after 2-0 win at S...
5     Prime Minister and his family are enjoying an ...
6     NBA star calls for black and Hispanic communit...
7     London Midland service had been pulling into T...
8     Tony Pulis believes Saido Berahino should look...
9     Black box data from Flight 370 could be analyz...
10    The distortion is the result of the Mercator m...
11    Alan Dershowitz has filed defamation suits aga...
12    Sarah Ernhart, the owner of Sarah Beth Photogr...
13    Manchester United beat Southampton 2-1 at St M...
14    U.S. corporations merge with foreign companies...
15    Half of Brits admit to checking work e-mails w...
16    Dencia's comment is hypocritical considering s...
17    Ex-military chiefs suggested .\nstepping u

In [37]:
text_summaries = []
for x in range(20):
    summary = str(model_summaries[x])
    summary = summary[2:-2]
    print(summary)
    text_summaries.append(summary)


Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.
The complaint alleges Ralph Mata arranged to pay assassins to kill rival drug dealers. Mata, also known as "The Milk Man," worked for the Miami-Dade Police Department. He is scheduled to appear in federal court on Wednesday.
Craig Eccleston-Todd, 27, was driving home from a night at a pub. He was so distracted he veered across the road and crashed into Rachel Titley's car. Miss Titley died in hospital after suffering serious injuries to her brain. Eccleston-Todd has been jailed for six years following the crash.
The EU must be ready to accept sanctions are a two-way street, writes Alexei Navalny. He says the EU should hold fire to Russia's peripatetic business community. Europe needs to deal w

In [38]:
text_summaries

['Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.',
 'The complaint alleges Ralph Mata arranged to pay assassins to kill rival drug dealers. Mata, also known as "The Milk Man," worked for the Miami-Dade Police Department. He is scheduled to appear in federal court on Wednesday.',
 "Craig Eccleston-Todd, 27, was driving home from a night at a pub. He was so distracted he veered across the road and crashed into Rachel Titley's car. Miss Titley died in hospital after suffering serious injuries to her brain. Eccleston-Todd has been jailed for six years following the crash.",
 "The EU must be ready to accept sanctions are a two-way street, writes Alexei Navalny. He says the EU should hold fire to Russia's peripatetic business community. Europe n

In [23]:
for x in range(20):
    model_summary = model1.predict(df['source_text'][x])
    
    model_summaries.append(model_summary)
    print(model_summaries[x])

['Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.']
['The complaint alleges Ralph Mata arranged to pay assassins to kill rival drug dealers. Mata, also known as "The Milk Man," worked for the Miami-Dade Police Department. He is scheduled to appear in federal court on Wednesday.']
["Craig Eccleston-Todd, 27, was driving home from a night at a pub. He was so distracted he veered across the road and crashed into Rachel Titley's car. Miss Titley died in hospital after suffering serious injuries to her brain. Eccleston-Todd has been jailed for six years following the crash."]
["The EU must be ready to accept sanctions are a two-way street, writes Alexei Navalny. He says the EU should hold fire to Russia's peripatetic business community. Europe n

In [31]:
model_summaries

[['Bishop John Folda of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion.'],
 ['The complaint alleges Ralph Mata arranged to pay assassins to kill rival drug dealers. Mata, also known as "The Milk Man," worked for the Miami-Dade Police Department. He is scheduled to appear in federal court on Wednesday.'],
 ["Craig Eccleston-Todd, 27, was driving home from a night at a pub. He was so distracted he veered across the road and crashed into Rachel Titley's car. Miss Titley died in hospital after suffering serious injuries to her brain. Eccleston-Todd has been jailed for six years following the crash."],
 ["The EU must be ready to accept sanctions are a two-way street, writes Alexei Navalny. He says the EU should hold fire to Russia's peripatetic business community. E

In [27]:
pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [40]:
summary_model

0     Bishop John Folda, of North Dakota, is taking ...
1     Criminal complaint: Cop used his role to help ...
2     Craig Eccleston-Todd, 27, had drunk at least t...
3     Nina dos Santos says Europe must be ready to a...
4     Fleetwood top of League One after 2-0 win at S...
5     Prime Minister and his family are enjoying an ...
6     NBA star calls for black and Hispanic communit...
7     London Midland service had been pulling into T...
8     Tony Pulis believes Saido Berahino should look...
9     Black box data from Flight 370 could be analyz...
10    The distortion is the result of the Mercator m...
11    Alan Dershowitz has filed defamation suits aga...
12    Sarah Ernhart, the owner of Sarah Beth Photogr...
13    Manchester United beat Southampton 2-1 at St M...
14    U.S. corporations merge with foreign companies...
15    Half of Brits admit to checking work e-mails w...
16    Dencia's comment is hypocritical considering s...
17    Ex-military chiefs suggested .\nstepping u

In [41]:
from rouge import Rouge


rouge = Rouge()
rouge.get_scores(summary_model, text_summaries, avg=True, ignore_empty=True)

{'rouge-1': {'r': 0.4038325363293443,
  'p': 0.3614023707110006,
  'f': 0.3720948721339845},
 'rouge-2': {'r': 0.17805848610778394,
  'p': 0.15135582343087273,
  'f': 0.15888454875420022},
 'rouge-l': {'r': 0.38618202759524545,
  'p': 0.3461092591006893,
  'f': 0.35594335935826094}}